In [1]:
import os
import cv2
import imghdr
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20212\2719461471.py:3: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
data_dir = 'data'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [4]:
for image_class in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, image_class)
    if os.path.isdir(class_dir):
        for image in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image)
            try:
                img = cv2.imread(image_path)
                tip = imghdr.what(image_path)
                if tip not in image_exts:
                    print('Image not in ext list {}'.format(image_path))
                    os.remove(image_path)
            except Exception as e:
                print('Issue with image {}'.format(image_path))

Issue with image data\bullet\.ipynb_checkpoints


In [5]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=(256, 256), batch_size=32)
classes = data

Found 389 files belonging to 3 classes.


In [6]:
num_classes = len(data.class_names)

In [7]:
data = data.map(lambda x, y: (x / 255, tf.one_hot(y, num_classes)))

In [8]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 16)        0

In [11]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2) + 1
test_size = int(len(data) * 0.1) + 1

In [12]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [13]:
logdir = 'logs'

In [14]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [15]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
9/9 [==============================] - 15s 1s/step - loss: 2.7404 - accuracy: 0.3299 - val_loss: 1.0682 - val_accuracy: 0.4688
Epoch 2/20
9/9 [==============================] - 15s 1s/step - loss: 1.0519 - accuracy: 0.4653 - val_loss: 0.9900 - val_accuracy: 0.5104
Epoch 3/20
9/9 [==============================] - 13s 1s/step - loss: 0.9484 - accuracy: 0.5972 - val_loss: 1.0146 - val_accuracy: 0.5208
Epoch 4/20
9/9 [==============================] - 13s 1s/step - loss: 0.9223 - accuracy: 0.5521 - val_loss: 0.9442 - val_accuracy: 0.5000
Epoch 5/20
9/9 [==============================] - 13s 1s/step - loss: 0.8924 - accuracy: 0.5833 - val_loss: 0.7555 - val_accuracy: 0.7292
Epoch 6/20
9/9 [==============================] - 12s 1s/step - loss: 0.7796 - accuracy: 0.6979 - val_loss: 0.5835 - val_accuracy: 0.7917
Epoch 7/20
9/9 [==============================] - 12s 1s/step - loss: 0.6780 - accuracy: 0.7639 - val_loss: 0.5963 - val_accuracy: 0.7812
Epoch 8/20
9/9 [==================

In [16]:
test_metrics = model.evaluate(test)
print("Test loss:", test_metrics[0])
print("Test accuracy:", test_metrics[1])

1/1 [==============================] - 2s 2s/step - loss: 0.0439 - accuracy: 1.0000
Test loss: 0.04390560835599899
Test accuracy: 1.0


In [17]:
image_path = 'bullettest.jpg'
img = cv2.imread(image_path)

In [18]:
resize = tf.image.resize(img, (256, 256)) / 255.0
resize = tf.expand_dims(resize, 0)

In [19]:
predictions = model.predict(resize)

1/1 [==============================] - 0s 198ms/step


In [20]:
predicted_class_index = tf.argmax(predictions, axis=1)[0]

In [21]:
class_names = classes.class_names

In [22]:
predicted_class_name = class_names[predicted_class_index]

In [23]:
print(f"Predicted class: {predicted_class_name}")

Predicted class: bullet


In [26]:
tf.saved_model.save(model, 'saved_model')

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


In [24]:
model.save("my_model")


INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [25]:
tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model my_model /tjs

SyntaxError: cannot assign to expression (466742034.py, line 1)

In [26]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model my_model /path/to/output_directory


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Lenovo\Desktop\SE factory\Final Project\medtech\tensorflow\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "C:\Users\Lenovo\Desktop\SE factory\Final Project\medtech\tensorflow\Lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "C:\Users\Lenovo\Desktop\SE factory\Final Project\medtech\tensorflow\Lib\site-packages\tensorflowjs\converters\__init__.py", line 21, in <module>
    from tensorflowjs.converters.converter import convert
  File "C:\Users\Lenovo\Desktop\SE factory\Final Project\medtech\tensorflow\Lib\site-packages\tensorflowjs\converters\converter.py", line 37, in <module>
    from tensorflowjs.converters import tf_saved_model_conversion_v2
  File "C:\Users\Lenovo\Desktop\SE factory\Final Project\medtech\tensorflow\Lib\site-packages\tenso

In [27]:
model.save('my_model.h5')

C:\Users\Lenovo\Desktop\SE factory\Final Project\medtech\tensorflow\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [29]:
tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model my_model /tjs

SyntaxError: cannot assign to expression (466742034.py, line 1)